---
# Import Libraries

In [1]:
import os
import sys

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [3]:
sys.path.append("../../../deep-learning-dna")
sys.path.append("../")
sys.path.append("../../../deep-learning-dna/common")

In [4]:
import wandb

In [5]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import math
import string
import time

from Attention import Set_Transformer
from common.models import dnabert
from common import dna
from lmdbm import Lmdb
from common.data import DnaSequenceGenerator, DnaLabelType, DnaSampleGenerator, find_dbs
import wandb

import tf_utilities as tfu

In [6]:
strategy = tfu.devices.select_gpu(0, use_dynamic_memory=True)

---
# Load Data

In [7]:
#Import pretrained model
api = wandb.Api()
model_path = api.artifact("sirdavidludwig/dnabert-pretrain/dnabert-pretrain-64dim:latest").download()
pretrained_model = dnabert.DnaBertModel.load(model_path)
pretrained_model.load_weights(model_path + "/model.h5")
pretrained_model

wandb:   5 of 5 files downloaded.  


In [8]:
#Load datafiles
dataset_path = api.artifact("sirdavidludwig/nachusa-dna/nachusa-dna:latest").download()
samples = find_dbs(dataset_path)

wandb: Downloading large artifact nachusa-dna:latest, 4079.09MB. 420 files... 
wandb:   420 of 420 files downloaded.  
Done. 0:0:13.8


In [9]:
len(samples)

210

---
# Create Dataset

In [10]:
split_ratios = [0.8, 0.2]
set_len = 1000
sequence_len = 150
kmer = 3
batch_size = [20,5]
batches_per_epoch = 20
augument = True
labels = DnaLabelType.SampleIds
seed = 0
rng = np.random.default_rng(seed)
random_samples = samples.copy()

In [11]:
rng.shuffle(random_samples)

In [12]:
trimmed_samples, (train_dataset, val_dataset) = DnaSampleGenerator.split(samples=random_samples[0:5], split_ratios=split_ratios, subsample_length=set_len, sequence_length=sequence_len, kmer=kmer,batch_size=batch_size,batches_per_epoch=batches_per_epoch,augment=augument,labels=labels, rng=rng)

In [13]:
t = time.time()
train_dataset[0][0]
time.time() - t

0.14112544059753418

--- 
# Batch Parameters

In [14]:
block_size = 200
max_set_len = set_len
max_files = len(train_dataset.samples)
max_files

5

In [15]:
if block_size-2 > set_len:
    raise ValueError("Block size should not be bigger than sequence length")

In [16]:
print(max_set_len)

1000


---
# Create Embeddings

In [17]:
#Create 8 dimensional embeddings
pretrained_encoder = dnabert.DnaBertEncoderModel(pretrained_model.base)
pretrained_encoder.trainable = False

In [18]:
class Create_Embeddings(keras.layers.Layer):
    def __init__(self, encoder):
        super(Create_Embeddings, self).__init__()
        self.encoder = encoder
        
    def subbatch_predict(self, model, batch, subbatch_size, concat=lambda old, new: tf.concat((old, new), axis=0)):
        def predict(i, result=None):
            n = i + subbatch_size
            pred = tf.stop_gradient(model(batch[i:n]))
            if result is None:
                return [n, pred]
            return [n, concat(result, pred)]
        i, result = predict(0)
        batch_size = tf.shape(batch)[0]
        i, result = tf.while_loop(
            cond=lambda i, _: i < batch_size,
            body=predict,
            loop_vars=[i, result],
            parallel_iterations=1)
        return result
    
    def modify_data_for_input(self, data):
        batch_size = tf.shape(data)[0]
        subsample_size = tf.shape(data)[1]
        flat_data = tf.reshape(data, (batch_size*subsample_size, -1))
        encoded = self.subbatch_predict(self.encoder, flat_data, 128)
        result = tf.reshape(encoded, (batch_size, subsample_size, -1))
        return result
    
    def call(self, data):
        return  self.modify_data_for_input(data)

---
# Cache Memory

In [19]:
def Cache_Memory(current_state, previous_state, memory_length):
    
    if memory_length is None or memory_length == 0:
        return None
    else:
        if previous_state is None:
            new_mem = current_state[:, -memory_length:, :]
        else:
            new_mem = tf.concat(
                    [previous_state, current_state], 1)[:, -memory_length:, :]

    return tf.stop_gradient(new_mem)

---
# Attention

In [20]:
class Attention(keras.Model):
    def __init__(self, num_induce, embed_dim, num_heads, use_layernorm, pre_layernorm, use_keras_mha):
        super(Attention, self).__init__()
        
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha
        
        if self.num_induce == 0:       
            self.attention = (Set_Transformer.SetAttentionBlock(embed_dim=self.embed_dim, num_heads=self.num_heads, use_layernorm=self.use_layernorm,pre_layernorm=self.pre_layernorm,use_keras_mha=self.use_keras_mha))
        else:
            self.attention = Set_Transformer.InducedSetAttentionBlock(embed_dim=self.embed_dim, num_heads=self.num_heads, num_induce=self.num_induce, use_layernorm=self.use_layernorm, pre_layernorm=self.pre_layernorm, use_keras_mha=self.use_keras_mha)
    
    
    def call(self, data, mems):
                
            attention = self.attention([data, mems])
                
            return attention

---
# XL Block

In [21]:
class TransformerXLBlock(tf.keras.layers.Layer):
    def __init__(self,
                 num_induce, 
                 embed_dim,
                 num_heads,
                 use_layernorm,
                 pre_layernorm,
                 use_keras_mha,):

        super(TransformerXLBlock, self).__init__()
        
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha
        
        self.attention = Attention
        
        self.attention_layer = self.attention(self.num_induce, self.embed_dim, self.num_heads, self.use_layernorm, self.pre_layernorm, self.use_keras_mha)

   
    def call(self,
             content_stream,
             state=None):
        
        attention_output = self.attention_layer(content_stream, state)

        return attention_output

---
# Transformer XL

In [22]:
class TransformerXL(tf.keras.layers.Layer):
    def __init__(self,
                 mem_switched, 
                 num_layers,
                 num_induce,
                 embed_dim,
                 num_heads,
                 dropout_rate,
                 mem_len=None,
                 use_layernorm=True,
                 pre_layernorm=True, 
                 use_keras_mha=True):
        
        super(TransformerXL, self).__init__()

        self.mem_switched = mem_switched
        self.num_layers = num_layers
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate
        self.mem_len = mem_len
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha

        self.transformer_xl_layers = []
        
        for i in range(self.num_layers):
            self.transformer_xl_layers.append(
                    TransformerXLBlock(self.num_induce,
                                        self.embed_dim,
                                        self.num_heads,
                                        self.use_layernorm,
                                        self.pre_layernorm, 
                                        self.use_keras_mha))

        self.output_dropout = tf.keras.layers.Dropout(rate=self.dropout_rate)

    def call(self,
             content_stream,
             state=None):
        
        new_mems = []

        if state is None:
            state = [None] * self.num_layers
            
        for i in range(self.num_layers):
            if self.mem_switched == False:
                new_mems.append(Cache_Memory(content_stream, state[i], self.mem_len))
            
            transformer_xl_layer = self.transformer_xl_layers[i]
            
            transformer_xl_output = transformer_xl_layer(content_stream=content_stream,
                                                        state=state[i])
            
            content_stream = self.output_dropout(transformer_xl_output)
            
            if self.mem_switched == True:
                new_mems.append(Cache_Memory(content_stream, state[i], self.mem_len))

        output_stream = content_stream
        return output_stream, new_mems

---
# Xl Model Class

In [23]:
class XlModel(keras.Model):
    def __init__(self, mem_switched, max_files, encoder, block_size, max_set_len, num_induce, embed_dim, num_layers, num_heads, mem_len, dropout_rate, num_seeds, use_layernorm, pre_layernorm, use_keras_mha):
        super(XlModel, self).__init__()
        
        self.mem_switched = mem_switched
        self.max_files = max_files
        self.encoder = encoder
        self.block_size = block_size
        self.max_set_len = max_set_len
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.mem_len = mem_len
        self.dropout_rate = dropout_rate
        self.num_seeds = num_seeds
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha
        
        self.embedding_layer = keras.layers.TimeDistributed(self.encoder)

        self.linear_layer = keras.layers.Dense(self.embed_dim)
        
        self.transformer_xl = TransformerXL(self.mem_switched,
                                            self.num_layers,
                                             self.num_induce,
                                             self.embed_dim,
                                             self.num_heads,
                                             self.dropout_rate,
                                             self.mem_len,
                                             self.use_layernorm,
                                             self.pre_layernorm,
                                             self.use_keras_mha)
        

        self.pooling_layer = Set_Transformer.PoolingByMultiHeadAttention(num_seeds=self.num_seeds,embed_dim=self.embed_dim,num_heads=self.num_heads,use_layernorm=self.use_layernorm,pre_layernorm=self.pre_layernorm, use_keras_mha=self.use_keras_mha, is_final_block=True)
    
        self.reshape_layer = keras.layers.Reshape((self.embed_dim,))
   
        self.output_layer = keras.layers.Dense(self.max_files, activation=keras.activations.softmax)
        
    
    def call(self, x, training=None):        
 
        mems = tf.zeros((self.num_layers, tf.shape(x)[0], self.mem_len, self.embed_dim))
    
        # embeddings = self.embedding_layer(x)
        embeddings = x
        linear_transform = self.linear_layer(embeddings)

        for i in range(0, self.max_set_len, self.block_size):
            block = linear_transform[:,i:i+self.block_size]
            
            output, mems = self.transformer_xl(content_stream=block, state=mems)
        
        pooling = self.pooling_layer(output)

        reshape = self.reshape_layer(pooling)

        output = self.output_layer(reshape)          
        
        return output

---
# Xl Parameters

In [24]:
#Xl Parameters
encoder = pretrained_encoder
mem_switched = False
num_induce = 10
embed_dim = 64
num_layers = 4
num_heads = 4
mem_len = 200
dropout_rate = 0.01
num_seeds = 1
use_layernorm = True
pre_layernorm = True
use_keras_mha = True

---
# Create Models

In [25]:
model = XlModel(mem_switched, max_files, encoder, block_size, max_set_len, num_induce, embed_dim, num_layers, num_heads, mem_len, dropout_rate, num_seeds, use_layernorm, pre_layernorm, use_keras_mha)
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False), optimizer = keras.optimizers.Adam(1e-3), metrics = keras.metrics.SparseCategoricalAccuracy())

In [26]:
model(train_dataset[0][0])
names = [weight.name for layer in model.layers for weight in layer.weights]
weights = model.get_weights()

name_counts = {}

for name, weight in zip(names, weights):
    if name not in name_counts:
        name_counts[name] = 0
    name_counts[name] += 1
name_counts

{'embedding_with_class_token/embedding/embeddings:0': 2,
 'dense/kernel:0': 2,
 'dense/bias:0': 2,
 'dense_1/kernel:0': 2,
 'dense_1/bias:0': 2,
 'relative_transformer_block/layer_normalization/gamma:0': 2,
 'relative_transformer_block/layer_normalization/beta:0': 2,
 'relative_transformer_block/layer_normalization_1/gamma:0': 2,
 'relative_transformer_block/layer_normalization_1/beta:0': 2,
 'relative_transformer_block/relative_multi_head_attention/relative_embeddings:0': 2,
 'relative_transformer_block/relative_multi_head_attention/query/kernel:0': 2,
 'relative_transformer_block/relative_multi_head_attention/query/bias:0': 2,
 'relative_transformer_block/relative_multi_head_attention/key/kernel:0': 2,
 'relative_transformer_block/relative_multi_head_attention/key/bias:0': 2,
 'relative_transformer_block/relative_multi_head_attention/value/kernel:0': 2,
 'relative_transformer_block/relative_multi_head_attention/value/bias:0': 2,
 'relative_transformer_block/relative_multi_head_attent

In [27]:
model.save_weights("save.h5")

In [39]:
epochs=1

In [40]:
import time

In [41]:
t = time.time()
train_dataset[0][0]
time.time() - t

0.132155179977417

In [64]:
20000*149*64*4/1024**2

727.5390625

In [53]:
inputs = encoder.predict(tf.reshape(train_dataset[0][0], (-1, 148)), batch_size=1000).reshape((20, 1000, -1))

20/20 [==============================] - 5s 258ms/step


In [51]:
t = time.time()
model(inputs)
time.time() - t

0.2700774669647217

In [31]:
model.summary()

Model: "xl_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dna_bert_encoder_model (Dna  multiple                 683968    
 BertEncoderModel)                                               
                                                                 
 create__embeddings (Create_  multiple                 683968    
 Embeddings)                                                     
                                                                 
 dense_17 (Dense)            multiple                  4160      
                                                                 
 transformer_xl (Transformer  multiple                 600064    
 XL)                                                             
                                                                 
 pooling_by_multi_head_atten  multiple                 74752     
 tion (PoolingByMultiHeadAtt                              

In [32]:
history = model.fit(x=train_dataset, validation_data=val_dataset, epochs=epochs, verbose=1)


KeyboardInterrupt

